In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data2 = data[['text','sentiment']]

In [3]:
data3=data2[data.sentiment!='Neutral']

In [4]:
data2.text.values

array(['RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate',
       "RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…",
       'RT @TJMShow: No mention of Tamir Rice and the #GOPDebate was held in Cleveland? Wow.',
       ...,
       'RT @Lrihendry: #TedCruz As President, I will always tell the truth, and do what I said I would do.  #GOPDebates',
       'RT @JRehling: #GOPDebate Donald Trump says that he doesn\'t have time for political correctness. How does calling women "fat pigs" save him …',
       'RT @Lrihendry: #TedCruz headed into the Presidential Debates. GO TED!! \r\n\r\n#GOPDebates http://t.co/8S67pz8a4A'],
      dtype=object)

In [5]:
sentiment_label=data3.sentiment.factorize()
sentiment_label[:5]

(array([0, 0, 0, ..., 0, 1, 0], dtype=int64),
 Index(['Positive', 'Negative'], dtype='object'))

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tweet = data3.text.values # select tweets
tokenizer = Tokenizer(num_words=5000) # Create tokenizer with max numbers 5000
tokenizer.fit_on_texts(tweet) # initialize tokenizer on our tweets
vocab_size = len(tokenizer.word_index) + 1 # vocab size
encoded_docs = tokenizer.texts_to_sequences(tweet) # tweets applied to our tokenizer
padded_sequence = pad_sequences(encoded_docs, maxlen=200) # tweets with 200 len, for lstm

In [7]:
print(tokenizer.word_index)

{'the': 1, 'gopdebate': 2, 'rt': 3, 'gopdebates': 4, 'to': 5, 'is': 6, 'of': 7, 'a': 8, 'and': 9, 't': 10, 'co': 11, 'i': 12, 'http': 13, 'rwsurfergirl': 14, '🇺🇸': 15, '\r': 16, 'you': 17, 'it': 18, 'in': 19, 'for': 20, 'trump': 21, 'that': 22, 'fox': 23, 'on': 24, 'not': 25, 'this': 26, 'was': 27, 'about': 28, 'realdonaldtrump': 29, 'amp': 30, 'debate': 31, 'news': 32, 'from': 33, 'have': 34, 'at': 35, 'we': 36, 'but': 37, 'he': 38, 'me': 39, 'just': 40, 'be': 41, 'candidates': 42, 'are': 43, '…': 44, 'like': 45, 'last': 46, 'so': 47, 'they': 48, 'up': 49, 'https': 50, 'all': 51, 'megynkelly': 52, 'with': 53, 'gop': 54, "don't": 55, 'what': 56, 'people': 57, 'jeb': 58, 'who': 59, 'foxnews': 60, 'my': 61, 'how': 62, 'as': 63, 'if': 64, 'night': 65, 'no': 66, 'has': 67, 'bush': 68, 'their': 69, 'by': 70, 'when': 71, 'can': 72, 'do': 73, 'more': 74, 'out': 75, 'one': 76, 'should': 77, 'or': 78, "i'm": 79, 'his': 80, 'would': 81, 'chris': 82, 'him': 83, 'get': 84, 'think': 85, 'republican

In [8]:
print(tweet[0])
print(encoded_docs[0])

RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
[3, 364, 136, 2462, 1, 717, 2, 46, 65, 259, 43, 216, 7, 4382, 178, 1798, 19, 1341, 1429, 761, 13, 10, 11]


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding

embedding_vector_length = 32

model = Sequential()

model.add(Embedding(5000, embedding_vector_length, input_length=200) )
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 176,651
Trainable params: 176,651
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
type(padded_sequence)

numpy.ndarray

In [10]:
history = model.fit(padded_sequence,sentiment_label[0],
                  validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
 94/269 [=========>....................] - ETA: 13s - loss: 0.5414 - accuracy: 0.7803

KeyboardInterrupt: 

In [ ]:
test_word ="This is very happy"
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]